In [ ]:
import chardet

caminho_arquivo = "/home/juniorsilvadev/engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/PostgreSQL/Arquivos/V_OCORRENCIA_AMPLA.json"

# Lendo os primeiros bytes do arquivo para detectar o encoding
with open(caminho_arquivo, "rb") as f:
    raw_data = f.read(10000)  # Lê os primeiros 10.000 bytes
    result = chardet.detect(raw_data)
    encoding_detectado = result['encoding']

print(f"Encoding detectado: {encoding_detectado}")

Encoding detectado: UTF-8-SIG


In [2]:
import pandas as pd

df = pd.read_json(caminho_arquivo, encoding=encoding_detectado)
df.head(5)

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrência,Data_da_Ocorrencia,Hora_da_Ocorrência,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,7762,201803231711488,ICON TAXI AEREO LTDA E OUTRO,Incidente,2018-03-21,20:00:00,SÃO PAULO,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,...,0.0,0.0,0.0,AW109SP,H2T,A109,3175.0,8.0,AGUSTA,verdadeiro
1,7759,201803161337187,EJ ESCOLA DE AERONAUTICA CIVIL LTDA,Acidente,2018-03-14,15:30:00,MONTES CLAROS,MG,Sudeste,COMBUSTÍVEL,...,0.0,0.0,0.0,A152,L1P,C152,757.0,2.0,CESSNA AIRCRAFT,verdadeiro
2,7758,201802271904132,AEROTEX AVIACAO AGRICOLA LTDA.,Acidente,2018-01-26,12:20:00,INACIOLÂNDIA,GO,Centro-Oeste,EXCURSÃO DE PISTA,...,0.0,0.0,0.0,T188C,L1P,C188,1997.0,1.0,CESSNA AIRCRAFT,verdadeiro
3,7758,201802271904132,AEROTEX AVIACAO AGRICOLA LTDA.,Acidente,2018-01-26,12:20:00,INACIOLÂNDIA,GO,Centro-Oeste,INCURSÃO EM PISTA,...,0.0,0.0,0.0,T188C,L1P,C188,1997.0,1.0,CESSNA AIRCRAFT,verdadeiro
4,7757,201803191230508,REALIZAR ESCOLA DE AVIACAO CIVIL LTDA,Incidente Grave,2018-03-18,20:30:00,TORRES,RS,Sul,CONTATO ANORMAL COM A PISTA,...,0.0,0.0,0.0,PA-34-200,L2P,PA34,1814.0,6.0,PIPER AIRCRAFT,verdadeiro


In [3]:
colunas = ['Numero_da_Ocorrencia', 'Classificacao_da_Ocorrência', 'Data_da_Ocorrencia', 'Municipio', 'UF', 'Regiao', 'Nome_do_Fabricante']
df = df[colunas]

In [4]:
df.rename(columns={'Classificacao_da_Ocorrência': 'Classificacao_da_Ocorrencia'}, inplace=True)
df.columns = df.columns.str.strip() # Remover espaços extras

In [5]:
print(df.columns.tolist())
df.head()

['Numero_da_Ocorrencia', 'Classificacao_da_Ocorrencia', 'Data_da_Ocorrencia', 'Municipio', 'UF', 'Regiao', 'Nome_do_Fabricante']


,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT


In [6]:
import psycopg2

# Parâmetros de conexão
dbname   = 'anac_db'
user     = 'postgres'
password = 'root'
host     = 'localhost'
port     = '5432'

try:
    # Criar uma conexão
    conexao = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)

    cursor = conexao.cursor() # Criar um cursor deixa manipular os dados
    print("Conexão bem-sucedida ao PostgreSQL!")
    
    conexao.commit() # Validar alterações que agente fez e subir para o banco de dados 
except Exception as e:
    print('Error ao conectar com banco de dados:', e)

Conexão bem-sucedida ao PostgreSQL!


In [7]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS anac (
        Numero_da_Ocorrencia int,
        Classificacao_da_Ocorrencia VARCHAR(50),
        Data_da_Ocorrencia DATE,
        Municipio VARCHAR(50),
        UF VARCHAR(30),
        Regiao VARCHAR(50),
        Nome_do_Fabricante VARCHAR(50)
    );
"""

try:
    cursor.execute(create_table_query)
    conexao.commit()
    print('Tabela criada com sucesso')
except Exception as e:
    print('Error ao criar a tabela', e)
    conexao.rollback()

Tabela criada com sucesso


In [8]:
cursor.execute("TRUNCATE TABLE anac")

In [9]:
insert_query = """
    INSERT INTO anac (
        Numero_da_Ocorrencia, Classificacao_da_Ocorrencia, Data_da_Ocorrencia, Municipio, UF, Regiao, Nome_do_Fabricante
    ) VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

# Carga de Dados
try:
    for index, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Numero_da_Ocorrencia'],
            row['Classificacao_da_Ocorrencia'],
            row['Data_da_Ocorrencia'],
            row['Municipio'],
            row['UF'],
            row['Regiao'],
            row['Nome_do_Fabricante']
        ))
    
    conexao.commit()
    print('Dados inseridos com sucesso')
        
except Exception as e:
    print('Error ao inserir dados:', e)
    conexao.rollback()
finally:
    cursor.close()
    conexao.close()

Dados inseridos com sucesso
